# PINN (series)

In [ ]:
from setproctitle import setproctitle
setproctitle("PINN (series)")

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "2"

import time
import numpy as np 
import matplotlib.pyplot as plt
from pathlib import Path
import glob

In [ ]:
b_bottom_paths = os.path.expanduser('~/workspace/_data/NOAA12673/b_bottom')
Nz = 160
spatial_norm = 160
b_norm = 2500

In [ ]:
total_iterations = 50000
log_interval = 1000

series_iteration = 2000
series_log_interval = 100

num_neurons = 256
num_layers = 8

w_ff = 1
w_div = 1
# w_bc_init = 1000
decay_iterations = 25000

# lr_init = 5e-4
# lr_final = 5e-5
# lr_decay_iterations = 50000

batch_size = 10000
num_worker = 2

In [ ]:
from cmspinn.pinn_nf2_old import NF2Trainer

In [ ]:
meta_path = None

for b_bottom_path in sorted(glob.glob(os.path.join(b_bottom_paths, '*.npy'))):
    b_bottom_date = os.path.basename(b_bottom_path)[9:-4]
    output_path = os.path.join(Path(b_bottom_path).parent.parent, f'PINN/{b_bottom_date}')
    
    with open(b_bottom_path, 'rb') as f:
        b_bottom = np.load(f)

    Nx, Ny, _ = b_bottom.shape
    
    final_model_path = os.path.join(output_path, 'model_final.pt')
    if os.path.exists(final_model_path):
        meta_path = final_model_path
        continue
    
    if meta_path is None:
        trainer = NF2Trainer(output_path, b_bottom, Nz, spatial_norm, b_norm,
                             meta_path=None, dim=num_neurons, w_div=w_div, w_ff=w_ff,
                             decay_iterations=decay_iterations)

        start = time.time()
        trainer.train(total_iterations, batch_size, log_interval, log_interval, num_workers=num_worker)
        runtime = time.time() - start
        trainer.logger.info(f'Runtime --> total: {runtime:.2f}sec ({(runtime/(total_iterations-1)*1000):.2f}ms/iter.)')
        meta_path = os.path.join(output_path, 'model_final.pt')
    else:
        trainer = NF2Trainer(output_path, b_bottom, Nz, spatial_norm, b_norm,
                             meta_path=meta_path, dim=num_neurons, w_div=w_div, w_ff=w_ff)

        start = time.time()
        trainer.train(series_iteration, batch_size, series_log_interval, series_log_interval, num_workers=num_worker)
        runtime = time.time() - start
        trainer.logger.info(f'Runtime --> total: {runtime:.2f}sec ({(runtime/(total_iterations-1)*1000):.2f}ms/iter.)')
        meta_path = os.path.join(output_path, 'model_final.pt')

Configuration:
dim: 256, w_div: 1.000000, w_ff: 1.000000, decay_iterations: 25000, potential: True, vector_potential: False, 
Using device: cuda (gpus 1) ['NVIDIA RTX A6000']
Training: 100%|██████████| 50000/50000 [1:01:45<00:00, 13.49it/s]
[Iteration 050000/050000] [loss: 0.00548254] [loss_bc: 0.00492673; loss_div: 0.00041049; loss_ff: 0.00014532] [w_bc: 1.000000, LR: 0.000050]
Runtime --> total: 3706.13sec (74.12ms/iter.)
Configuration:
dim: 256, w_div: 1.000000, w_ff: 1.000000, decay_iterations: None, potential: True, vector_potential: False, 
Using device: cuda (gpus 1) ['NVIDIA RTX A6000']
Potential Boundary: 100%|██████████| 1/1 [00:00<00:00,  1.07it/s]
Loaded meta state: /userhome/jeon_mg/workspace/_data/NOAA12673/PINN/20170906_083600/model_final.pt
Training: 100%|██████████| 2000/2000 [02:27<00:00, 13.51it/s]
[Iteration 002000/002000] [loss: 0.00477512] [loss_bc: 0.00426055; loss_div: 0.00036107; loss_ff: 0.00015350] [w_bc: 1.000000, LR: 0.000050]
Runtime --> total: 148.15sec (